# Tutorial 4: Reducibility Candidates

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-normalization/blob/main/notebooks/04-reducibility-candidates.ipynb)

> "The secret is to define 'normalizing' by induction on types, not terms." — Varen Tholl

## The Normalization Problem

We want to prove: **Every well-typed term reduces to a normal form.**

The naive approach — induction on terms — doesn't work. Consider the application rule:

```
If Γ ⊢ M : A → B and Γ ⊢ N : A, then Γ ⊢ M N : B
```

To show M N normalizes, we might try:
- M normalizes (by induction)
- N normalizes (by induction)
- Therefore M N normalizes?

But M N might be larger than M! The induction breaks.

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

# Load reducibility candidates dataset
rc_df = pd.read_csv(BASE_URL + "reducibility_candidates.csv")

rc_df[['type_constructor', 'definition', 'used_for']].head(4)

## The Key Insight

Varen Tholl developed the **reducibility method** in Year 945 (EV-945-005), inspired by Brennis Mund's original technique:

**Induct on types, not terms.**

For each type A, define a set of terms CR(A) called "reducibility candidates" or "strongly normalizing terms of type A". Then prove:

1. All terms in CR(A) are strongly normalizing
2. Every well-typed term of type A is in CR(A)

In [ ]:
# The reducibility method structure
method = pd.DataFrame({
    'step': [1, 2, 3],
    'goal': [
        'Define CR(A) for each type A',
        'Prove CR(A) ⊆ SN for all A',
        'Prove Γ ⊢ M : A implies M ∈ CR(A)'
    ],
    'technique': [
        'Induction on type structure',
        'Follows from definition',
        'Induction on typing derivation'
    ]
})

method

## Defining Reducibility Candidates

### Base Types

For a base type like `Nat`:

```
CR(Nat) = { M : Nat | M is strongly normalizing }
```

Simple: a term of base type is reducible if it normalizes.

In [ ]:
# Base type reducibility
print("CR(Nat) = { M : Nat | M strongly normalizes }")
print("")
print("Examples:")
print("  zero ∈ CR(Nat)  ✓  (already normal)")
print("  succ zero ∈ CR(Nat)  ✓  (already normal)")
print("  (λx:Nat.x) zero ∈ CR(Nat)  ✓  (normalizes to zero)")

### Arrow Types

For function types `A → B`, the definition is subtle:

```
CR(A → B) = { M : A → B | for all N ∈ CR(A), M N ∈ CR(B) }
```

A function is reducible if applying it to any reducible argument gives a reducible result.

In [ ]:
# Arrow type reducibility
print("CR(A → B) = { M : A → B | ∀N ∈ CR(A). M N ∈ CR(B) }")
print("")
print("Example: Is (λx:Nat.succ x) ∈ CR(Nat → Nat)?")
print("")
print("  For any N ∈ CR(Nat), we need (λx:Nat.succ x) N ∈ CR(Nat)")
print("  (λx:Nat.succ x) N → succ N")
print("  If N normalizes, so does succ N")
print("  Therefore (λx:Nat.succ x) ∈ CR(Nat → Nat)  ✓")

### Dependent Types (Pi and Sigma)

For dependent function types `Π(x:A).B(x)`:

```
CR(Π(x:A).B) = { M | for all N ∈ CR(A), M N ∈ CR(B[N/x]) }
```

The result type depends on the argument value — so the reducibility set varies.

In [ ]:
# View all reducibility candidate definitions
rc_df[['type_constructor', 'definition', 'closure_properties']]

## Closure Properties

For the proof to work, CR(A) must satisfy certain **closure properties**:

1. **CR1**: CR(A) ⊆ SN (all reducible terms normalize)
2. **CR2**: If M ∈ CR(A) and M → M', then M' ∈ CR(A) (closed under reduction)
3. **CR3**: If M is neutral and all one-step reducts of M are in CR(A), then M ∈ CR(A)

In [ ]:
# Closure properties
closure = pd.DataFrame({
    'property': ['CR1', 'CR2', 'CR3'],
    'statement': [
        'CR(A) ⊆ SN',
        'M ∈ CR(A) and M → M\' implies M\' ∈ CR(A)',
        'M neutral and reducts in CR(A) implies M ∈ CR(A)'
    ],
    'meaning': [
        'Reducibility implies normalization',
        'Closed under reduction',
        'Neutral terms are base case'
    ]
})

closure

## Neutral Terms

A **neutral term** is one that can't reduce on its own:
- Variables: `x`
- Applications of variables: `x y`
- Eliminator applications: `rec_nat n base step` where n is neutral

Neutral terms are the "base case" of reducibility. They're reducible if all their reducts are.

In [ ]:
# Neutral terms
neutral = pd.DataFrame({
    'term': ['x', 'x y', 'rec_nat x base step', '(λx.M) N'],
    'is_neutral': [True, True, True, False],
    'reason': [
        'Variable, can\'t reduce',
        'Variable at head, can\'t reduce',
        'Eliminator on neutral, can\'t reduce',
        'Redex, CAN reduce'
    ]
})

neutral

## The Main Lemma

The key lemma is:

**Fundamental Theorem of Reducibility**: If Γ ⊢ M : A and all variables in Γ are replaced by reducible terms, then M is reducible.

Proof by induction on the typing derivation:

- **Variables**: x is reducible by assumption (it was replaced by a reducible term)
- **Lambda**: λx:A.M is reducible if applying it to reducibles gives reducibles
- **Application**: M N is reducible by definition of CR(A → B)

In [ ]:
# The proof structure
proof_cases = pd.DataFrame({
    'case': ['Variable', 'Lambda', 'Application', 'Constructor', 'Eliminator'],
    'form': ['x', 'λx:A.M', 'M N', 'zero, succ n', 'rec_nat n b s'],
    'why_reducible': [
        'Hypothesis (substituted reducible)',
        'Apply to reducible, get reducible (IH)',
        'M in CR(A→B), N in CR(A), so M N in CR(B)',
        'Subterms reducible, constructor is value',
        'Reduces to reducible by IH'
    ]
})

proof_cases

## Why This Works

The reducibility method succeeds because:

1. **Induction on types** — types have simpler structure than terms
2. **Semantic definition** — CR(A → B) is defined in terms of behavior, not syntax
3. **Closure properties** — ensure the definition is well-behaved

The result: Normalization follows from the structure of the type system itself.

In [ ]:
# Compare to termination arguments
term_df = pd.read_csv(BASE_URL + "termination_arguments.csv")

term_df[term_df['proof_technique'] == 'reducibility_candidates'][['type_system', 'terminates', 'why_terminates']]

## Summary

| Step | What We Do | Why It Works |
|------|-----------|---------------|
| Define CR(A) | Reducible terms of type A | Induction on type structure |
| Prove CR(A) ⊆ SN | Reducible implies normalizing | Follows from definition |
| Prove typed ⊆ CR | All typed terms are reducible | Induction on derivation |
| Conclude | All typed terms normalize | CR(A) ⊆ SN |

This technique, pioneered by Brennis for simple types and extended by Tholl to dependent types, remains the standard proof of strong normalization.

---

**Next Tutorial:** Termination and Recursion — Why recursion is dangerous